In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import cross_validate
import re
%matplotlib inline

In [2]:
#importing the data

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
full_data = [train, test]

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


# Feature Engineering and data cleaning

In [6]:
trainX =train.drop(['Survived'],axis=1)

In [7]:
trainY=train['Survived']

In [8]:
#Feature Engineering for Both datasets should be done, but only considering train data values

In [9]:
AllData = pd.concat([trainX, test])

In [10]:
AllData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


In [11]:
#Missing Value treatment for Age, Fare using median of the training set

In [12]:
train['Age'] = train['Age'].fillna(train['Age'].median())
train['Fare'] = train['Fare'].fillna(train['Fare'].median())

In [13]:
#Using the same values to AllData

In [14]:
AllData['Fare'] = AllData['Fare'].fillna(trainX['Fare'].median())

In [15]:
AllData['Age'] = AllData['Age'].fillna(trainX['Age'].median())

In [16]:
#binnnig Age variable, binning should be based on Training data range

In [17]:
train['AgeCat'] = pd.qcut(train.Age, q=4)

In [18]:
train['AgeCat'].value_counts()

(22.0, 28.0]     308
(0.419, 22.0]    231
(35.0, 80.0]     217
(28.0, 35.0]     135
Name: AgeCat, dtype: int64

In [19]:
#using the Above bin categories to categorize Alldata

In [20]:
AllData['AgeCat']=AllData['Age'].apply(lambda x: 0 if x<=22 else (1 if x<=28 else (2 if x<=35 else 3)))

In [21]:
#binnnig Fare variable, binning should be based on Training data range

In [22]:
train['Fare'] = pd.qcut(train.Fare, q=4)

In [23]:
train['Fare'].value_counts()

(7.91, 14.454]     224
(-0.001, 7.91]     223
(31.0, 512.329]    222
(14.454, 31.0]     222
Name: Fare, dtype: int64

In [24]:
#using the Above bin categories of Fare to categorize Alldata

In [25]:
AllData['FareCat']=AllData['Fare'].apply(lambda x: 0 if x<=7.91 else (1 if x<=14.454 else (2 if x<=31.0 else 3)))

In [26]:
#Missing Value treatment for Embarked, replacing it with it's mode 'S'

In [27]:
AllData['Embarked'] = AllData['Embarked'].fillna('S')

In [28]:
AllData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1309 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1309 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1309 non-null   object 
 11  AgeCat       1309 non-null   int64  
 12  FareCat      1309 non-null   int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 143.2+ KB


In [29]:
#Missing Value treatment for Cabin
#Assuming cabin variable is for people who had a cabin, and null is for people who didnt have a cabin, new variable Has_Cabin is created

In [30]:
train['Has_Cabin'] = ~train.Cabin.isnull()

In [31]:
print (train[['Has_Cabin', 'Survived']].groupby(['Has_Cabin'], as_index=False).mean())

   Has_Cabin  Survived
0      False  0.299854
1       True  0.666667


In [32]:
#since the survival rates are very different, we use the Has_Cabin in the features. And Drop the feature Cabin


In [33]:
AllData['Has_Cabin'] = ~AllData.Cabin.isnull()
AllData = AllData.drop(['Cabin'],axis=1)


In [34]:
AllData.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,AgeCat,FareCat,Has_Cabin
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0,0,False
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,3,3,True
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1,1,False
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,2,3,True
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,2,1,False


In [35]:
#Creating Family_size using SibSp and Parch variables, checking the survival rates for each count

In [36]:
train['Family_size'] = train['SibSp'] + train['Parch']
print (train[['Family_size', 'Survived']].groupby(['Family_size'], as_index=False).mean())

   Family_size  Survived
0            0  0.303538
1            1  0.552795
2            2  0.578431
3            3  0.724138
4            4  0.200000
5            5  0.136364
6            6  0.333333
7            7  0.000000
8           10  0.000000


In [37]:
#Binning the family size if alone =0, if has less than 4 members=1 or more than four members=2

In [38]:
train['FamilyCount']=train['Family_size'].apply(lambda x: 0 if x==0 else (1 if x<4 else 2))

In [39]:
print (train[['FamilyCount', 'Survived']].groupby(['FamilyCount'], as_index=False).mean())

   FamilyCount  Survived
0            0  0.303538
1            1  0.578767
2            2  0.161290


In [40]:
#Since the survival rate for above is clearly different, we use as the new feature in the Dataset

In [41]:
AllData['Family_size'] = AllData['SibSp'] + AllData['Parch']
AllData['FamilyCount']=AllData['Family_size'].apply(lambda x: 0 if x==0 else (1 if x<4 else 2))

In [42]:
#Dropping SibSp, Parch and Family_Size variables

In [43]:
AllData = AllData.drop(['SibSp','Parch','Family_size'],axis=1)

In [44]:
#Information From the Name

In [45]:
AllData['Title'] = AllData.Name.apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1))

In [46]:
AllData['Title'].value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Ms            2
Mlle          2
Major         2
Dona          1
Don           1
Sir           1
Jonkheer      1
Mme           1
Lady          1
Capt          1
Countess      1
Name: Title, dtype: int64

In [47]:
#replacing french words and rare terms

AllData['Title'] = AllData['Title'].replace({'Mlle':'Miss', 'Mme':'Mrs', 'Ms':'Miss'})
AllData['Title'] = AllData['Title'].replace(['Don', 'Dona', 'Rev', 'Dr',
                                            'Major', 'Lady', 'Sir', 'Col', 'Capt', 'Countess', 'Jonkheer'],'Rare')

In [48]:
AllData['Title'].value_counts()

Mr        757
Miss      264
Mrs       198
Master     61
Rare       29
Name: Title, dtype: int64

In [49]:
AllData

,PassengerId,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,AgeCat,FareCat,Has_Cabin,FamilyCount,Title
0,1,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,0,0,False,1,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,3,3,True,1,Mrs
2,3,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,1,1,False,0,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,2,3,True,1,Mrs
4,5,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,2,1,False,0,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,28.0,A.5. 3236,8.0500,S,1,1,False,0,Mr
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,PC 17758,108.9000,C,3,3,True,0,Rare
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,SOTON/O.Q. 3101262,7.2500,S,3,0,False,0,Mr
416,1308,3,"Ware, Mr. Frederick",male,28.0,359309,8.0500,S,1,1,False,0,Mr


In [50]:
#Removing Columns that are not neccessary

In [51]:
AllDataCopy=AllData

In [52]:
AllData = AllData.drop(['PassengerId','Name','Age','Ticket','Fare'],axis=1)

In [53]:
AllData

,Pclass,Sex,Embarked,AgeCat,FareCat,Has_Cabin,FamilyCount,Title
0,3,male,S,0,0,False,1,Mr
1,1,female,C,3,3,True,1,Mrs
2,3,female,S,1,1,False,0,Miss
3,1,female,S,2,3,True,1,Mrs
4,3,male,S,2,1,False,0,Mr
...,...,...,...,...,...,...,...,...
413,3,male,S,1,1,False,0,Mr
414,1,female,C,3,3,True,0,Rare
415,3,male,S,3,0,False,0,Mr
416,3,male,S,1,1,False,0,Mr


In [54]:
le = LabelEncoder()
NewAllData = AllData.apply(le.fit_transform)

In [55]:
#Diffrenciating Train and Test Data

Newtrain = NewAllData.iloc[:891]
Newtest = NewAllData.iloc[891:]

In [56]:
#Cross Validation set up

def run_kfold(clf):
    kf = KFold(891, n_splits=10)
    outcomes = []
    fold = 0
    for train_index, test_index in kf:
        fold += 1
        Xtrain, Xtest = Newtrain.values[train_index], train_X.values[test_index]
        ytrain, ytest = trainY.values[train_index], train_y.values[test_index]
        clf.fit(Xtrain, ytrain)
        predictions = clf.predict(Xtest)
        accuracy = accuracy_score(ytest, predictions)
        outcomes.append(accuracy)
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

# Model Selection

In [57]:
#DecisionTreeClassifier

In [58]:
model = DecisionTreeClassifier(random_state=0)

In [59]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = KFold(n_splits=10)


results = cross_validate(estimator=model,X=Newtrain,y=trainY,cv=kfold,scoring=scoring)

In [60]:
print("Accuracy =" , np.mean(results['test_accuracy']))
print("Precision =" , np.mean(results['test_precision']))
print("Recall =" , np.mean(results['test_recall']))
print("f1 score =" , np.mean(results['test_f1_score']))

Accuracy = 0.8002372034956304
Precision = 0.7673826145094595
Recall = 0.6750176245764481
f1 score = 0.7136399301727454


In [61]:
#Naive Bayes Classifier

In [62]:
model2 = GaussianNB()

In [63]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = KFold(n_splits=10)


results = cross_validate(estimator=model2,X=Newtrain,y=trainY,cv=kfold,scoring=scoring)

In [64]:
print("Accuracy =" , np.mean(results['test_accuracy']))
print("Precision =" , np.mean(results['test_precision']))
print("Recall =" , np.mean(results['test_recall']))
print("f1 score =" , np.mean(results['test_f1_score']))

Accuracy = 0.7408114856429463
Precision = 0.6346164248568196
Recall = 0.7546064711873536
f1 score = 0.6855010314582237


In [77]:
#Rule Based Classifier - Using Ripper Algrorithm


In [78]:
pip install wittgenstein

Note: you may need to restart the kernel to use updated packages.


In [79]:
import wittgenstein as lw

In [80]:
clf = lw.RIPPER()

In [81]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = KFold(n_splits=10)


results = cross_validate(estimator=clf,X=Newtrain,y=trainY,cv=kfold,scoring=scoring)

In [83]:
print("Accuracy =" , np.mean(results['test_accuracy']))
print("Precision =" , np.mean(results['test_precision']))
print("Recall =" , np.mean(results['test_recall']))
print("f1 score =" , np.mean(results['test_f1_score']))

Accuracy = 0.7834456928838952
Precision = 0.8567964725333146
Recall = 0.5227261254467137
f1 score = 0.6426371908387338


##  Predicting Data for the test Data

In [65]:
#Using Desicion Tree

DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(Newtrain,trainY)
main_Y=DTC.predict(Newtest)

ids = test['PassengerId']
predictions = main_Y
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('submissions.csv', index = False)

In [68]:
#Using Naive Bayes

GNB = GaussianNB()
GNB.fit(Newtrain,trainY)
main_Y1=GNB.predict(Newtest)

ids = test['PassengerId']
predictions = main_Y1
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('submissions_GNB.csv', index = False)

In [84]:
#Using Rule Based Ripper Algorithm

clf = lw.RIPPER()
clf.fit(Newtrain,trainY)
main_Y2=clf.predict(Newtest)

ids = test['PassengerId']
predictions = main_Y2
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('submissions_rulebased.csv', index = False)